In [1]:
import random
import gym
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
import MyNN

In [25]:
class PGAgent:
    def __init__(self, state_size, action_size, gamma=0.95,learning_rate=0.01):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma  # discount rate
        self.learning_rate = learning_rate
        self.model = self._build_model()

    def _build_model(self):
        model = MyNN.MyNN(self.state_size)
        model.add(24, 'ReLU')
        model.add(24, 'ReLU')
        model.add(1, 'Sigmoid')
        model.compile('Cross entropy', 'Adam')
        return model

    def act(self, state):
        result = self.model.forward(state)[0][0]
        return 1 if result > np.random.random() else 0

In [3]:
def play_game(agent, render=False):
    state = env.reset()
    memory = np.array([0,0,0,0]).reshape(4,1)
    actions = np.array([])
    rewards = np.array([])
    for time in range(500):
        if render:
            env.render()
        memory = np.hstack((memory, state.reshape(4,1)))
        action = agent.act(state)
        actions = np.append(actions, action)
        state, reward, done, _ = env.step(action)
        state = state.reshape(4,1)
        if done:
            reward = -10
        rewards = np.append(rewards, reward)
        if done or time == 498:
            break
    return memory[,1:], actions, rewards, time

In [4]:
def running_reward(actions, rewards, gamma):
    result = []
    run_rew = -10
    for action, reward in list(zip(actions, rewards))[::-1]:
        run_rew = run_rew*gamma + reward
        result.append([action, run_rew])
    return np.vstack(result[::-1])

In [9]:
def play_n_games(agent, n=20):
    X_batch = np.array([0,0,0,0]).reshape(4,1)
    y_batch = np.array([0,0])
    results = []
    for i in range(n):
        a, b, c, d = play_game(agent)
        X_batch = np.vstack((X_batch, a))
        y_batch = np.vstack((y_batch, running_reward(b, c, agent.gamma)))
        results.append(d)
    return X_batch[0,1:], y_batch[1:], results

In [26]:
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = PGAgent(state_size, action_size, learning_rate=0.01)

[2018-01-07 13:55:11,120] Making new env: CartPole-v1


In [27]:
%%time
results = []
minimums = []
maximums = []
for t in range(1, 1001):
    X_batch, y_batch, result = play_n_games(agent, n=10)
    results.append(np.mean(result))
    minimums.append(np.min(result))
    maximums.append(np.max(result))
    print(t, np.mean(result), np.var(result), batch_size, result)
    if np.mean(result) == 498:
        break
    agent.model.optimize(X_batch, Y_batch, lr=agent.learning_rate, num_epochs=1, report_cost=False)

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [24]:
state = env.reset()
agent.model.forward(state.reshape(4,1))[0][0]

0.50501993749317786